In [1]:
from utils.helpers import *
from utils.preprocess_data import *

In [ ]:
import pandas as pd

data = pd.read_csv("./data/augment_data.csv", encoding="utf8")
data.incorrect = data.incorrect.str.strip()
data.correct = data.correct.str.strip()

In [3]:
data.head(5)

,incorrect,correct
0,"Hkacs với những lần trươ1c, tàu họ đã được tàu...","Khác với những lần trước, tàu họ đã được tàu t..."
1,Cầu thủ mang về bàn ghỡ hòa cho đội khách Tcho...,Cầu thủ mang về bàn gỡ hòa cho đội khách Tchoy...
2,Sau đó tôi dô4 thế nào nó cũng chỉ nghi một ch...,Sau đó tôi dỗ thế nào nó cũng chỉ ngồi một chỗ...
3,ĐB của Đồng bằng sụng Cửu Long này xúc động gl...,ĐB của Đồng bằng sông Cửu Long này xúc động nó...
4,Mạch điều jgifren trung tâm.,Mạch điều khiển trung tâm.


In [2]:
def process_labels(correct, incorrect):
    _, labels = perfect_align(incorrect, correct, 3)
    final_label = []
    skip_flag = False
    for idx in range(len(labels)):
        label = labels[idx]
        if idx + 1 < len(labels):
            next_label = labels[idx+1]
            if label[0].startswith("KEEP") and next_label[0].startswith("APPEND"):
                final_label.append([next_label[0], label[1]])
                skip_flag = True
                if idx + 1 == len(labels) - 1:
                    break
            elif label[0].startswith("APPEND") and next_label[0].startswith("APPEND"):
                if final_label:
                    tmp_label, tmp_e = final_label[-1]
                    tmp_label = tmp_label + next_label[0][-1]
                    final_label[-1] = [tmp_label, tmp_e]
                    if idx + 1 == len(labels) - 1:
                        break
                else:
                    tmp_label, tmp_e = label
                    tmp_label = tmp_label + next_label[0][-1]
                    final_label.append([tmp_label, tmp_e])
                    skip_flag = True
            elif skip_flag:
                skip_flag = False
            else:
                final_label.append(label)
                skip_flag = False
        else:
            final_label.append(label)

    if len(final_label) == len(incorrect) + 1:
        if final_label[0][1] != final_label[1][1]:
            print(f"Length labels not equal for sentence: {incorrect}")
            raise Exception

    elif len(final_label) != len(incorrect):
        print(f"Length labels not equal for sentence: {incorrect}")
        raise Exception

    return [label[0] for label in final_label]

In [5]:
corrects = data.correct.values.tolist()
incorrects = data.incorrect.values.tolist()

In [6]:
def extract_general_tag(label):
    for i in range(len(label)):
        if label[i].startswith("REPLACE") or label[i].startswith("APPEND"):
            label[i] = label[i].split("_")[0]
    return label

In [ ]:
from tqdm import tqdm
edit_tags = []
general_tags = []
for correct, incorrect in tqdm(zip(corrects[:500], incorrects[:500])):
    edit_tag = process_labels(correct, incorrect)
    general_tag = extract_general_tag(edit_tag)
    assert len(edit_tag)==len(general_tag), "LENGTH NOT EQUAL WHILE EXTRACTING"
    edit_tags.append(edit_tag)
    general_tags.append(general_tag)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "incorrect": incorrects[:500],
    "correct": corrects[:500],
    "edit_tags": edit_tags,
    "general_tags": general_tags
})
df.to_csv("test_dataset.csv", encoding="utf8")

In [3]:
process_labels("giống", "rống")

['APPEND_g', 'REPLACE_i', 'KEEP', 'KEEP', 'KEEP']